In [13]:
# import random
# import math
# import numbers
# import collections
# import numpy as np
# import torch
# from PIL import Image, ImageOps
# try:
#     import accimage
# except ImportError:
#     accimage = None


# class Compose(object):
#     """Composes several transforms together.
#     Args:
#         transforms (list of ``Transform`` objects): list of transforms to compose.
#     Example:
#         >>> transforms.Compose([
#         >>>     transforms.CenterCrop(10),
#         >>>     transforms.ToTensor(),
#         >>> ])
#     """

#     def __init__(self, transforms):
#         self.transforms = transforms

#     def __call__(self, img):
#         for t in self.transforms:
#             img = t(img)
#         return img

#     def randomize_parameters(self):
#         for t in self.transforms:
#             t.randomize_parameters()


# class ToTensor(object):
#     """Convert a ``PIL.Image`` or ``numpy.ndarray`` to tensor.
#     Converts a PIL.Image or numpy.ndarray (H x W x C) in the range
#     [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
#     """

#     def __init__(self, norm_value=255):
#         self.norm_value = norm_value

#     def __call__(self, pic):
#         """
#         Args:
#             pic (PIL.Image or numpy.ndarray): Image to be converted to tensor.
#         Returns:
#             Tensor: Converted image.
#         """
#         if isinstance(pic, np.ndarray):
#             # handle numpy array
#             img = torch.from_numpy(pic.transpose((2, 0, 1)))
#             # backward compatibility
#             return img.float().div(self.norm_value)

#         if accimage is not None and isinstance(pic, accimage.Image):
#             nppic = np.zeros(
#                 [pic.channels, pic.height, pic.width], dtype=np.float32)
#             pic.copyto(nppic)
#             return torch.from_numpy(nppic)

#         # handle PIL Image
#         if pic.mode == 'I':
#             img = torch.from_numpy(np.array(pic, np.int32, copy=False))
#         elif pic.mode == 'I;16':
#             img = torch.from_numpy(np.array(pic, np.int16, copy=False))
#         else:
#             img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
#         # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK
#         if pic.mode == 'YCbCr':
#             nchannel = 3
#         elif pic.mode == 'I;16':
#             nchannel = 1
#         else:
#             nchannel = len(pic.mode)
#         img = img.view(pic.size[1], pic.size[0], nchannel)
#         # put it from HWC to CHW format
#         # yikes, this transpose takes 80% of the loading time/CPU
#         img = img.transpose(0, 1).transpose(0, 2).contiguous()
#         if isinstance(img, torch.ByteTensor):
#             return img.float().div(self.norm_value)
#         else:
#             return img

#     def randomize_parameters(self):
#         pass


# class Normalize(object):
#     """Normalize an tensor image with mean and standard deviation.
#     Given mean: (R, G, B) and std: (R, G, B),
#     will normalize each channel of the torch.*Tensor, i.e.
#     channel = (channel - mean) / std
#     Args:
#         mean (sequence): Sequence of means for R, G, B channels respecitvely.
#         std (sequence): Sequence of standard deviations for R, G, B channels
#             respecitvely.
#     """

#     def __init__(self, mean, std):
#         self.mean = mean
#         self.std = std

#     def __call__(self, tensor):
#         """
#         Args:
#             tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
#         Returns:
#             Tensor: Normalized image.
#         """
#         # TODO: make efficient
#         for t, m, s in zip(tensor, self.mean, self.std):
#             t.sub_(m).div_(s)
#         return tensor

#     def randomize_parameters(self):
#         pass


# class Scale(object):
#     """Rescale the input PIL.Image to the given size.
#     Args:
#         size (sequence or int): Desired output size. If size is a sequence like
#             (w, h), output size will be matched to this. If size is an int,
#             smaller edge of the image will be matched to this number.
#             i.e, if height > width, then image will be rescaled to
#             (size * height / width, size)
#         interpolation (int, optional): Desired interpolation. Default is
#             ``PIL.Image.BILINEAR``
#     """

#     def __init__(self, size, interpolation=Image.BILINEAR):
#         assert isinstance(size,
#                           int) or (isinstance(size, collections.Iterable) and
#                                    len(size) == 2)
#         self.size = size
#         self.interpolation = interpolation

#     def __call__(self, img):
#         """
#         Args:
#             img (PIL.Image): Image to be scaled.
#         Returns:
#             PIL.Image: Rescaled image.
#         """
#         if isinstance(self.size, int):
#             w, h = img.size
#             if (w <= h and w == self.size) or (h <= w and h == self.size):
#                 return img
#             if w < h:
#                 ow = self.size
#                 oh = int(self.size * h / w)
#                 return img.resize((ow, oh), self.interpolation)
#             else:
#                 oh = self.size
#                 ow = int(self.size * w / h)
#                 return img.resize((ow, oh), self.interpolation)
#         else:
#             return img.resize(self.size, self.interpolation)

#     def randomize_parameters(self):
#         pass


# class CenterCrop(object):
#     """Crops the given PIL.Image at the center.
#     Args:
#         size (sequence or int): Desired output size of the crop. If size is an
#             int instead of sequence like (h, w), a square crop (size, size) is
#             made.
#     """

#     def __init__(self, size):
#         if isinstance(size, numbers.Number):
#             self.size = (int(size), int(size))
#         else:
#             self.size = size

#     def __call__(self, img):
#         """
#         Args:
#             img (PIL.Image): Image to be cropped.
#         Returns:
#             PIL.Image: Cropped image.
#         """
#         w, h = img.size
#         th, tw = self.size
#         x1 = int(round((w - tw) / 2.))
#         y1 = int(round((h - th) / 2.))
#         return img.crop((x1, y1, x1 + tw, y1 + th))

#     def randomize_parameters(self):
#         pass


# class CornerCrop(object):

#     def __init__(self, size, crop_position=None):
#         self.size = size
#         if crop_position is None:
#             self.randomize = True
#         else:
#             self.randomize = False
#         self.crop_position = crop_position
#         self.crop_positions = ['c', 'tl', 'tr', 'bl', 'br']

#     def __call__(self, img):
#         image_width = img.size[0]
#         image_height = img.size[1]

#         if self.crop_position == 'c':
#             th, tw = (self.size, self.size)
#             x1 = int(round((image_width - tw) / 2.))
#             y1 = int(round((image_height - th) / 2.))
#             x2 = x1 + tw
#             y2 = y1 + th
#         elif self.crop_position == 'tl':
#             x1 = 0
#             y1 = 0
#             x2 = self.size
#             y2 = self.size
#         elif self.crop_position == 'tr':
#             x1 = image_width - self.size
#             y1 = 0
#             x2 = image_width
#             y2 = self.size
#         elif self.crop_position == 'bl':
#             x1 = 0
#             y1 = image_height - self.size
#             x2 = self.size
#             y2 = image_height
#         elif self.crop_position == 'br':
#             x1 = image_width - self.size
#             y1 = image_height - self.size
#             x2 = image_width
#             y2 = image_height

#         img = img.crop((x1, y1, x2, y2))

#         return img

#     def randomize_parameters(self):
#         if self.randomize:
#             self.crop_position = self.crop_positions[random.randint(
#                 0,
#                 len(self.crop_positions) - 1)]


# class RandomHorizontalFlip(object):
#     """Horizontally flip the given PIL.Image randomly with a probability of 0.5."""

#     def __call__(self, img):
#         """
#         Args:
#             img (PIL.Image): Image to be flipped.
#         Returns:
#             PIL.Image: Randomly flipped image.
#         """
#         if self.p < 0.5:
#             return img.transpose(Image.FLIP_LEFT_RIGHT)
#         return img

#     def randomize_parameters(self):
#         self.p = random.random()


# class MultiScaleCornerCrop(object):
#     """Crop the given PIL.Image to randomly selected size.
#     A crop of size is selected from scales of the original size.
#     A position of cropping is randomly selected from 4 corners and 1 center.
#     This crop is finally resized to given size.
#     Args:
#         scales: cropping scales of the original size
#         size: size of the smaller edge
#         interpolation: Default: PIL.Image.BILINEAR
#     """

#     def __init__(self,
#                  scales,
#                  size,
#                  interpolation=Image.BILINEAR,
#                  crop_positions=['c', 'tl', 'tr', 'bl', 'br']):
#         self.scales = scales
#         self.size = size
#         self.interpolation = interpolation

#         self.crop_positions = crop_positions

#     def __call__(self, img):
#         min_length = min(img.size[0], img.size[1])
#         crop_size = int(min_length * self.scale)

#         image_width = img.size[0]
#         image_height = img.size[1]

#         if self.crop_position == 'c':
#             center_x = image_width // 2
#             center_y = image_height // 2
#             box_half = crop_size // 2
#             x1 = center_x - box_half
#             y1 = center_y - box_half
#             x2 = center_x + box_half
#             y2 = center_y + box_half
#         elif self.crop_position == 'tl':
#             x1 = 0
#             y1 = 0
#             x2 = crop_size
#             y2 = crop_size
#         elif self.crop_position == 'tr':
#             x1 = image_width - crop_size
#             y1 = 0
#             x2 = image_width
#             y2 = crop_size
#         elif self.crop_position == 'bl':
#             x1 = 0
#             y1 = image_height - crop_size
#             x2 = crop_size
#             y2 = image_height
#         elif self.crop_position == 'br':
#             x1 = image_width - crop_size
#             y1 = image_height - crop_size
#             x2 = image_width
#             y2 = image_height

#         img = img.crop((x1, y1, x2, y2))

#         return img.resize((self.size, self.size), self.interpolation)

#     def randomize_parameters(self):
#         self.scale = self.scales[random.randint(0, len(self.scales) - 1)]
#         self.crop_position = self.crop_positions[random.randint(
#             0,
#             len(self.scales) - 1)]


# class MultiScaleRandomCrop(object):

#     def __init__(self, scales, size, interpolation=Image.BILINEAR):
#         self.scales = scales
#         self.size = size
#         self.interpolation = interpolation

#     def __call__(self, img):
#         min_length = min(img.size[0], img.size[1])
#         crop_size = int(min_length * self.scale)

#         image_width = img.size[0]
#         image_height = img.size[1]

#         x1 = self.tl_x * (image_width - crop_size)
#         y1 = self.tl_y * (image_height - crop_size)
#         x2 = x1 + crop_size
#         y2 = y1 + crop_size

#         img = img.crop((x1, y1, x2, y2))

#         return img.resize((self.size, self.size), self.interpolation)

#     def randomize_parameters(self):
#         self.scale = self.scales[random.randint(0, len(self.scales) - 1)]
#         self.tl_x = random.random()
#         self.tl_y = random.random()

In [14]:
# import random
# import math


# class Compose(object):

#     def __init__(self, transforms):
#         self.transforms = transforms

#     def __call__(self, target):
#         dst = []
#         for t in self.transforms:
#             dst.append(t(target))
#         return dst


# class ClassLabel(object):

#     def __call__(self, target):
#         return target['label']


# class VideoID(object):

#     def __call__(self, target):
#         return target['video_id']

In [15]:
# import random
# import math


# class LoopPadding(object):

#     def __init__(self, size):
#         self.size = size

#     def __call__(self, frame_indices):
#         out = frame_indices

#         for index in out:
#             if len(out) >= self.size:
#                 break
#             out.append(index)

#         return out


# class TemporalBeginCrop(object):
#     """Temporally crop the given frame indices at a beginning.
#     If the number of frames is less than the size,
#     loop the indices as many times as necessary to satisfy the size.
#     Args:
#         size (int): Desired output size of the crop.
#     """

#     def __init__(self, size):
#         self.size = size

#     def __call__(self, frame_indices):
#         out = frame_indices[:self.size]

#         for index in out:
#             if len(out) >= self.size:
#                 break
#             out.append(index)

#         return out


# class TemporalCenterCrop(object):
#     """Temporally crop the given frame indices at a center.
#     If the number of frames is less than the size,
#     loop the indices as many times as necessary to satisfy the size.
#     Args:
#         size (int): Desired output size of the crop.
#     """

#     def __init__(self, size):
#         self.size = size

#     def __call__(self, frame_indices):
#         """
#         Args:
#             frame_indices (list): frame indices to be cropped.
#         Returns:
#             list: Cropped frame indices.
#         """

#         center_index = len(frame_indices) // 2
#         begin_index = max(0, center_index - (self.size // 2))
#         end_index = min(begin_index + self.size, len(frame_indices))

#         out = frame_indices[begin_index:end_index]

#         for index in out:
#             if len(out) >= self.size:
#                 break
#             out.append(index)

#         return out


# class TemporalRandomCrop(object):
#     """Temporally crop the given frame indices at a random location.
#     If the number of frames is less than the size,
#     loop the indices as many times as necessary to satisfy the size.
#     Args:
#         size (int): Desired output size of the crop.
#     """

#     def __init__(self, size):
#         self.size = size

#     def __call__(self, frame_indices):
#         """
#         Args:
#             frame_indices (list): frame indices to be cropped.
#         Returns:
#             list: Cropped frame indices.
#         """

#         rand_end = max(0, len(frame_indices) - self.size - 1)
#         begin_index = random.randint(0, rand_end)
#         end_index = min(begin_index + self.size, len(frame_indices))

#         out = frame_indices[begin_index:end_index]

#         for index in out:
#             if len(out) >= self.size:
#                 break
#             out.append(index)

#         return out

In [26]:
import torch
import torch.utils.data as data
from PIL import Image
import os
import functools
import json
import copy
import math

from utils import load_value_file


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')


def accimage_loader(path):
    try:
        import accimage
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)


def get_default_image_loader():
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader
    else:
        return pil_loader


def video_loader(video_dir_path, frame_indices, image_loader):
    video = []
    for i in frame_indices:
        image_path = os.path.join(video_dir_path, 'image_{:05d}.jpg'.format(i))
        if os.path.exists(image_path):
            video.append(image_loader(image_path))
        else:
            return video

    return video


def get_default_video_loader():
    image_loader = get_default_image_loader()
    return functools.partial(video_loader, image_loader=image_loader)


def load_annotation_data(data_file_path):
    with open(data_file_path, 'r') as data_file:
        return json.load(data_file)


def get_class_labels(data):
    class_names = []
    index = 0
    for node1 in data['taxonomy']:
        is_leaf = True
        for node2 in data['taxonomy']:
            if node2['parentId'] == node1['nodeId']:
                is_leaf = False
                break
        if is_leaf:
            class_names.append(node1['nodeName'])

    class_labels_map = {}

    for i, class_name in enumerate(class_names):
        class_labels_map[class_name] = i

    return class_labels_map


def get_video_names_and_annotations(data, subset):
    video_names = []
    annotations = []

    for key, value in data['database'].items():
        this_subset = value['subset']
        if this_subset == subset:
            if subset == 'testing':
                video_names.append('v_{}'.format(key))
            else:
                video_names.append('v_{}'.format(key))
                annotations.append(value['annotations'])

    return video_names, annotations


def modify_frame_indices(video_dir_path, frame_indices):
    modified_indices = []
    for i in frame_indices:
        image_path = os.path.join(video_dir_path, 'image_{:05d}.jpg'.format(i))
        if not os.path.exists(image_path):
            return modified_indices
        modified_indices.append(i)
    return modified_indices


#m_captions
def make_dataset(root_path, annotation_path, subset, n_samples_for_each_video,
                 sample_duration):
    
    caption_path = 'data/captions/train.json'
    captions_json = json.load(open(caption_path, 'r'))
    
    data = load_annotation_data(annotation_path)
    video_names, annotations = get_video_names_and_annotations(data, subset)
    
    #class_to_idx = get_class_labels(data)
    idx_to_class = {}
    #for name, label in class_to_idx.items():
        #idx_to_class[label] = name

    dataset = []
    for i in range(len(video_names)):
        if i % 1000 == 0:
            print('dataset loading [{}/{}]'.format(i, len(video_names)))

        video_path = os.path.join(root_path, video_names[i])
        if not os.path.exists(video_path):
            continue

        fps_file_path = os.path.join(video_path, 'fps')
        fps = load_value_file(fps_file_path)
        video_id = video_names[i]
        if video_id not in captions_json:
            print("Warning: caption not found for ",video_id)
            continue
        else:
            print("Caption found for ",video_id)

        for j in range(len(captions_json[video_id]['timestamps'])):
            begin_t = math.ceil(captions_json[video_id]['timestamps'][j][0] * fps)
            end_t = math.ceil(captions_json[video_id]['timestamps'][j][1] * fps)
            if begin_t == 0:
                begin_t = 1
            n_frames = end_t - begin_t

            sample = {
                'video': video_path,
                'segment': [begin_t, end_t],
                'fps': fps,
                'video_id': video_names[i][2:],
            }
            if(j < len(captions_json[video_id]['timestamps'])):
                sample['caption'] = captions_json[video_id]['sentences'][j]
            else:
                sample['caption'] = -1


#             if len(annotations) != 0:
#                 sample['label'] = class_to_idx[annotation['label']]
#             else:
#                 sample['label'] = -1

            if n_samples_for_each_video == 1:
                frame_indices = list(range(begin_t, end_t))
                frame_indices = modify_frame_indices(sample['video'],
                                                     frame_indices)
                if len(frame_indices) < 16:
                    continue
                sample['frame_indices'] = frame_indices
                dataset.append(sample)
            else:
                if n_samples_for_each_video > 1:
                    step = max(1,
                               math.ceil((n_frames - 1 - sample_duration) /
                                         (n_samples_for_each_video - 1)))
                else:
                    step = sample_duration
                for j in range(begin_t, end_t, step):
                    sample_j = copy.deepcopy(sample)
                    frame_indices = list(range(j, j + sample_duration))
                    frame_indices = modify_frame_indices(
                        sample_j['video'], frame_indices)
                    if len(frame_indices) < 16:
                        continue
                    sample_j['frame_indices'] = frame_indices
                    dataset.append(sample_j)

    return dataset, idx_to_class

#m_dataset
# def make_dataset(root_path, annotation_path, subset, n_samples_for_each_video,
#                  sample_duration):
#     data = load_annotation_data(annotation_path)
#     video_names, annotations = get_video_names_and_annotations(data, subset)
#     class_to_idx = get_class_labels(data)
#     idx_to_class = {}
#     for name, label in class_to_idx.items():
#         idx_to_class[label] = name

#     dataset = []
#     for i in range(len(video_names)):
#         if i % 1000 == 0:
#             print('dataset loading [{}/{}]'.format(i, len(video_names)))

#         video_path = os.path.join(root_path, video_names[i])
#         if not os.path.exists(video_path):
#             continue

#         fps_file_path = os.path.join(video_path, 'fps')
#         fps = load_value_file(fps_file_path)

#         for annotation in annotations[i]:
#             begin_t = math.ceil(annotation['segment'][0] * fps)
#             end_t = math.ceil(annotation['segment'][1] * fps)
#             if begin_t == 0:
#                 begin_t = 1
#             n_frames = end_t - begin_t

#             sample = {
#                 'video': video_path,
#                 'segment': [begin_t, end_t],
#                 'fps': fps,
#                 'video_id': video_names[i][2:]
#             }
#             if len(annotations) != 0:
#                 sample['label'] = class_to_idx[annotation['label']]
#             else:
#                 sample['label'] = -1

#             if n_samples_for_each_video == 1:
#                 frame_indices = list(range(begin_t, end_t))
#                 frame_indices = modify_frame_indices(sample['video'],
#                                                      frame_indices)
#                 if len(frame_indices) < 16:
#                     continue
#                 sample['frame_indices'] = frame_indices
#                 dataset.append(sample)
#             else:
#                 if n_samples_for_each_video > 1:
#                     step = max(1,
#                                math.ceil((n_frames - 1 - sample_duration) /
#                                          (n_samples_for_each_video - 1)))
#                 else:
#                     step = sample_duration
#                 for j in range(begin_t, end_t, step):
#                     sample_j = copy.deepcopy(sample)
#                     frame_indices = list(range(j, j + sample_duration))
#                     frame_indices = modify_frame_indices(
#                         sample_j['video'], frame_indices)
#                     if len(frame_indices) < 16:
#                         continue
#                     sample_j['frame_indices'] = frame_indices
#                     dataset.append(sample_j)

#     return dataset, idx_to_class


def get_end_t(video_path):
    file_names = os.listdir(video_path)
    image_file_names = [x for x in file_names if 'image' in x]
    image_file_names.sort(reverse=True)
    return int(image_file_names[0][6:11])


def make_untrimmed_dataset(root_path, annotation_path, subset,
                           n_samples_for_each_video, sample_duration):
    data = load_annotation_data(annotation_path)
    video_names, _ = get_video_names_and_annotations(data, subset)
    class_to_idx = get_class_labels(data)
    captions_path = 'data/captions/train.json'
    captions_json = json.load(open(captions_path, 'r'))
    
    idx_to_class = {}
    for name, label in class_to_idx.items():
        idx_to_class[label] = name

    dataset = []
    for i in range(len(video_names)):
        if i % 1000 == 0:
            print('dataset loading [{}/{}]'.format(i, len(video_names)))

        video_path = os.path.join(root_path, video_names[i])
        if not os.path.exists(video_path):
            continue

        fps_file_path = os.path.join(video_path, 'fps')
        fps = load_value_file(fps_file_path)

        begin_t = 1
        end_t = get_end_t(video_path)
        n_frames = end_t - begin_t
        
        sample = {
            'video': video_path,
            'segment': [begin_t, end_t],
            'fps': fps,
            'video_id': video_names[i][2:]
        }

        if n_samples_for_each_video >= 1:
            step = max(1,
                       math.ceil((n_frames - 1 - sample_duration) /
                                 (n_samples_for_each_video - 1)))
        else:
            step = sample_duration
        for j in range(begin_t, end_t, step):
            sample_j = copy.deepcopy(sample)
            frame_indices = list(range(j, j + sample_duration))
            frame_indices = modify_frame_indices(sample_j['video'],
                                                 frame_indices)
            if len(frame_indices) < 16:
                continue
            sample_j['frame_indices'] = frame_indices
            dataset.append(sample_j)

    return dataset, idx_to_class


class ActivityNet(data.Dataset):
    """
    Args:
        root (string): Root directory path.
        spatial_transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        temporal_transform (callable, optional): A function/transform that  takes in a list of frame indices
            and returns a transformed version
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an video given its path and frame indices.
     Attributes:
        classes (list): List of the class names.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(self,
                 root_path,
                 annotation_path,
                 subset,
                 is_untrimmed_setting=False,
                 n_samples_for_each_video=1,
                 spatial_transform=None,
                 temporal_transform=None,
                 target_transform=None,
                 sample_duration=16,
                 get_loader=get_default_video_loader):
        if is_untrimmed_setting:
            self.data, self.class_names = make_untrimmed_dataset(
                root_path, annotation_path, subset, n_samples_for_each_video,
                sample_duration)
        else:
            self.data, self.class_names = make_dataset(
                root_path, annotation_path, subset, n_samples_for_each_video,
                sample_duration)

        self.spatial_transform = spatial_transform
        self.temporal_transform = temporal_transform
        self.target_transform = target_transform
        self.loader = get_loader()

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        path = self.data[index]['video']

        frame_indices = self.data[index]['frame_indices']
        if self.temporal_transform is not None:
            frame_indices = self.temporal_transform(frame_indices)
        clip = self.loader(path, frame_indices)
        if self.spatial_transform is not None:
            self.spatial_transform.randomize_parameters()
            clip = [self.spatial_transform(img) for img in clip]
        clip = torch.stack(clip, 0).permute(1, 0, 2, 3)

        target = self.data[index]
        if self.target_transform is not None:
            target = self.target_transform(target)

        return clip, target

    def __len__(self):
        return len(self.data)

In [17]:
#m_opts
import argparse


def parse_opts():
    parser = argparse.ArgumentParser()
    print(type(parser))
    parser.add_argument(
        '--root_path',
        #default='/root/data/ActivityNet',
        default='data',
        type=str,
        help='Root directory path of data')
    parser.add_argument(
        '--video_path',
        #default='video_kinetics_jpg',
        default='jpg_videos',
        type=str,
        help='Directory path of Videos')
    parser.add_argument(
        '--annotation_path',
        #default='kinetics.json',
        default='activitynet.json',
        type=str,
        help='Annotation file path')
    parser.add_argument(
        '--result_path',
        default='results',
        type=str,
        help='Result directory path')
    parser.add_argument(
        '--dataset',
        default='activitynet',
        #default='kinetics',
        type=str,
        help='Used dataset (activitynet | kinetics | ucf101 | hmdb51)')
    parser.add_argument(
        '--n_classes',
        default=200,
        type=int,
        help=
        'Number of classes (activitynet: 200, kinetics: 400, ucf101: 101, hmdb51: 51)'
    )
    parser.add_argument(
        '--n_finetune_classes',
        default=400,
        type=int,
        help=
        'Number of classes for fine-tuning. n_classes is set to the number when pretraining.'
    )
    parser.add_argument(
        '--sample_size',
        default=112,
        type=int,
        help='Height and width of inputs')
    parser.add_argument(
        '--sample_duration',
        default=16,
        type=int,
        help='Temporal duration of inputs')
    parser.add_argument(
        '--initial_scale',
        default=1.0,
        type=float,
        help='Initial scale for multiscale cropping')
    parser.add_argument(
        '--n_scales',
        default=5,
        type=int,
        help='Number of scales for multiscale cropping')
    parser.add_argument(
        '--scale_step',
        default=0.84089641525,
        type=float,
        help='Scale step for multiscale cropping')
    parser.add_argument(
        '--train_crop',
        default='corner',
        type=str,
        help=
        'Spatial cropping method in training. random is uniform. corner is selection from 4 corners and 1 center.  (random | corner | center)'
    )
    parser.add_argument(
        '--learning_rate',
        default=0.1,
        type=float,
        help=
        'Initial learning rate (divided by 10 while training by lr scheduler)')
    parser.add_argument('--momentum', default=0.9, type=float, help='Momentum')
    parser.add_argument(
        '--dampening', default=0.9, type=float, help='dampening of SGD')
    parser.add_argument(
        '--weight_decay', default=1e-3, type=float, help='Weight Decay')
    parser.add_argument(
        '--mean_dataset',
        default='activitynet',
        type=str,
        help=
        'dataset for mean values of mean subtraction (activitynet | kinetics)')
    parser.add_argument(
        '--no_mean_norm',
        action='store_true',
        help='If true, inputs are not normalized by mean.')
    parser.set_defaults(no_mean_norm=False)
    parser.add_argument(
        '--std_norm',
        action='store_true',
        help='If true, inputs are normalized by standard deviation.')
    parser.set_defaults(std_norm=False)
    parser.add_argument(
        '--nesterov', action='store_true', help='Nesterov momentum')
    parser.set_defaults(nesterov=False)
    parser.add_argument(
        '--optimizer',
        default='sgd',
        type=str,
        help='Currently only support SGD')
    parser.add_argument(
        '--lr_patience',
        default=10,
        type=int,
        help='Patience of LR scheduler. See documentation of ReduceLROnPlateau.'
    )
    parser.add_argument(
        '--batch_size', default=1, type=int, help='Batch Size')
        #'--batch_size', default=128, type=int, help='Batch Size')
    
    parser.add_argument(
        '--n_epochs',
        default=3,
        type=int,
        help='Number of total epochs to run')
    parser.add_argument(
        '--begin_epoch',
        default=1,
        type=int,
        help=
        'Training begins at this epoch. Previous trained model indicated by resume_path is loaded.'
    )
    parser.add_argument(
        '--n_val_samples',
        default=3,
        type=int,
        help='Number of validation samples for each activity')
    parser.add_argument(
        '--resume_path',
        default='',
        type=str,
        help='Save data (.pth) of previous training')
    parser.add_argument(
        '--pretrain_path', default='', type=str, help='Pretrained model (.pth)')
    parser.add_argument(
        '--ft_begin_index',
        default=0,
        type=int,
        help='Begin block index of fine-tuning')
    parser.add_argument(
        '--no_train',
        action='store_true',
        help='If true, training is not performed.')
    parser.set_defaults(no_train=False)
    parser.add_argument(
        '--no_val',
        action='store_true',
        help='If true, validation is not performed.')
    parser.set_defaults(no_val=True)
    parser.add_argument(
        '--test', action='store_true', help='If true, test is performed.')
    parser.set_defaults(test=False)
    parser.add_argument(
        '--test_subset',
        default='val',
        type=str,
        help='Used subset in test (val | test)')
    parser.add_argument(
        '--scale_in_test',
        default=1.0,
        type=float,
        help='Spatial scale in test')
    parser.add_argument(
        '--crop_position_in_test',
        default='c',
        type=str,
        help='Cropping method (c | tl | tr | bl | br) in test')
    parser.add_argument(
        '--no_softmax_in_test',
        action='store_true',
        help='If true, output for each clip is not normalized using softmax.')
    parser.set_defaults(no_softmax_in_test=False)
    parser.add_argument(
        '--no_cuda', action='store_true', help='If true, cuda is not used.')
    parser.set_defaults(no_cuda=True)
    parser.add_argument(
        '--n_threads',
        default=4,
        type=int,
        help='Number of threads for multi-thread loading')
    parser.add_argument(
        '--checkpoint',
        default=10,
        type=int,
        help='Trained model is saved at every this epochs.')
    parser.add_argument(
        '--no_hflip',
        action='store_true',
        help='If true holizontal flipping is not performed.')
    parser.set_defaults(no_hflip=False)
    parser.add_argument(
        '--norm_value',
        default=1,
        type=int,
        help=
        'If 1, range of inputs is [0-255]. If 255, range of inputs is [0-1].')
    parser.add_argument(
        '--model',
        default='resnet',
        type=str,
        help='(resnet | preresnet | wideresnet | resnext | densenet | ')
    parser.add_argument(
        '--model_depth',
        default=10,
        type=int,
        help='Depth of resnet (10 | 18 | 34 | 50 | 101)')
    parser.add_argument(
        '--resnet_shortcut',
        default='B',
        type=str,
        help='Shortcut type of resnet (A | B)')
    parser.add_argument(
        '--wide_resnet_k', default=2, type=int, help='Wide resnet k')
    parser.add_argument(
        '--resnext_cardinality',
        default=32,
        type=int,
        help='ResNeXt cardinality')
    parser.add_argument(
        '--manual_seed', default=1, type=int, help='Manually set random seed')
    print("almost done")
    import sys
    sys.argv=['']
    del sys
    args = parser.parse_args()
    print("done")
    return args

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]


def conv3x3x3(in_planes, out_planes, stride=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)


def downsample_basic_block(x, planes, stride):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if isinstance(out.data, torch.cuda.FloatTensor):
        zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 sample_size,
                 sample_duration,
                 shortcut_type='B',
                 num_classes=400):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            3,
            64,
            kernel_size=7,
            stride=(1, 2, 2),
            padding=(3, 3, 3),
            bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=2)
        last_duration = int(math.ceil(sample_duration / 16))
        last_size = int(math.ceil(sample_size / 32))
        self.avgpool = nn.AvgPool3d(
            (last_duration, last_size, last_size), stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def get_fine_tuning_parameters(model, ft_begin_index):
    if ft_begin_index == 0:
        return model.parameters()

    ft_module_names = []
    for i in range(ft_begin_index, 5):
        ft_module_names.append('layer{}'.format(i))
    ft_module_names.append('fc')

    parameters = []
    for k, v in model.named_parameters():
        for ft_module in ft_module_names:
            if ft_module in k:
                parameters.append({'params': v})
                break
        else:
            parameters.append({'params': v, 'lr': 0.0})

    return parameters

#m_models

def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

In [19]:
import sys
sys.path.insert(0, 'models/')

import torch
from torch import nn

#import resnet, pre_act_resnet, wide_resnet, resnext, densenet
#m_generate

def generate_model(opt):
    assert opt.model in [
        'resnet', 'preresnet', 'wideresnet', 'resnext', 'densenet'
    ]

    if opt.model == 'resnet':
        assert opt.model_depth in [10, 18, 34, 50, 101, 152, 200]

        #from resnet import get_fine_tuning_parameters

        if opt.model_depth == 10:
            model = resnet10(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 18:
            model = resnet18(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 34:
            model = resnet34(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 50:
            model = resnet50(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 101:
            model = resnet101(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 152:
            model = resnet152(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 200:
            model = resnet200(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
    elif opt.model == 'wideresnet':
        assert opt.model_depth in [50]

        #from models.wide_resnet import get_fine_tuning_parameters

        if opt.model_depth == 50:
            model = resnet50(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                k=opt.wide_resnet_k,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
    elif opt.model == 'resnext':
        assert opt.model_depth in [50, 101, 152]

        #from models.resnext import get_fine_tuning_parameters

        if opt.model_depth == 50:
            model = resnext.resnet50(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                cardinality=opt.resnext_cardinality,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 101:
            model = resnext.resnet101(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                cardinality=opt.resnext_cardinality,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 152:
            model = resnext.resnet152(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                cardinality=opt.resnext_cardinality,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
    elif opt.model == 'preresnet':
        assert opt.model_depth in [18, 34, 50, 101, 152, 200]

        #from models.pre_act_resnet import get_fine_tuning_parameters

        if opt.model_depth == 18:
            model = pre_act_resnet.resnet18(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 34:
            model = pre_act_resnet.resnet34(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 50:
            model = pre_act_resnet.resnet50(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 101:
            model = pre_act_resnet.resnet101(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 152:
            model = pre_act_resnet.resnet152(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 200:
            model = pre_act_resnet.resnet200(
                num_classes=opt.n_classes,
                shortcut_type=opt.resnet_shortcut,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
    elif opt.model == 'densenet':
        assert opt.model_depth in [121, 169, 201, 264]

        #from models.densenet import get_fine_tuning_parameters

        if opt.model_depth == 121:
            model = densenet.densenet121(
                num_classes=opt.n_classes,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 169:
            model = densenet.densenet169(
                num_classes=opt.n_classes,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 201:
            model = densenet.densenet201(
                num_classes=opt.n_classes,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)
        elif opt.model_depth == 264:
            model = densenet.densenet264(
                num_classes=opt.n_classes,
                sample_size=opt.sample_size,
                sample_duration=opt.sample_duration)

    if not opt.no_cuda:
        model = model.cuda()
        model = nn.DataParallel(model, device_ids=None)

        if opt.pretrain_path:
            print('loading pretrained model {}'.format(opt.pretrain_path))
            pretrain = torch.load(opt.pretrain_path)
            assert opt.arch == pretrain['arch']

            model.load_state_dict(pretrain['state_dict'])

            if opt.model == 'densenet':
                model.module.classifier = nn.Linear(
                    model.module.classifier.in_features, opt.n_finetune_classes)
                model.module.classifier = model.module.classifier.cuda()
            else:
                model.module.fc = nn.Linear(model.module.fc.in_features,
                                            opt.n_finetune_classes)
                model.module.fc = model.module.fc.cuda()

            parameters = get_fine_tuning_parameters(model, opt.ft_begin_index)
            return model, parameters
    else:
        if opt.pretrain_path:
            print('loading pretrained model {}'.format(opt.pretrain_path))
            pretrain = torch.load(opt.pretrain_path)
            assert opt.arch == pretrain['arch']

            model.load_state_dict(pretrain['state_dict'])

            if opt.model == 'densenet':
                model.classifier = nn.Linear(
                    model.classifier.in_features, opt.n_finetune_classes)
            else:
                model.fc = nn.Linear(model.fc.in_features,
                                            opt.n_finetune_classes)

            parameters = get_fine_tuning_parameters(model, opt.ft_begin_index)
            return model, parameters

    return model, model.parameters()

In [20]:
def get_mean(norm_value=255, dataset='activitynet'):
    assert dataset in ['activitynet', 'kinetics']

    if dataset == 'activitynet':
        return [
            114.7748 / norm_value, 107.7354 / norm_value, 99.4750 / norm_value
        ]
    elif dataset == 'kinetics':
        # Kinetics (10 videos for each class)
        return [
            110.63666788 / norm_value, 103.16065604 / norm_value,
            96.29023126 / norm_value
        ]


def get_std(norm_value=255):
    # Kinetics (10 videos for each class)
    return [
        38.7568578 / norm_value, 37.88248729 / norm_value,
        40.02898126 / norm_value
    ]

In [21]:
#m_train
import torch
from torch.autograd import Variable
import time
import os
import sys

from utils import AverageMeter, calculate_accuracy


def train_epoch(epoch, data_loader, model, criterion, optimizer, opt,
                epoch_logger, batch_logger):
    print('train at epoch {}'.format(epoch))

    model.train()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()

    end_time = time.time()
    print("len: ",len(data_loader))
    for i, (inputs, targets) in enumerate(data_loader):
        
        print("debug: Train: targets : ",targets)


        data_time.update(time.time() - end_time)

        if not opt.no_cuda:
            targets = targets.cuda(async=True)
        inputs = Variable(inputs)
        targets = Variable(targets)
        print("debug: Train:inputTensor.size : ",inputs.size())
        outputs = model(inputs)
        print("debug: Train: outputTensor.size : ",outputs.size())


        loss = criterion(outputs, targets)
        acc = calculate_accuracy(outputs, targets)

        losses.update(loss.data[0], inputs.size(0))
        accuracies.update(acc, inputs.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        batch_logger.log({
            'epoch': epoch,
            'batch': i + 1,
            'iter': (epoch - 1) * len(data_loader) + (i + 1),
            'loss': losses.val,
            'acc': accuracies.val,
            'lr': optimizer.param_groups[0]['lr']
        })

        print('Epoch: [{0}][{1}/{2}]\t'
              'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
              'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
              'Acc {acc.val:.3f} ({acc.avg:.3f})'.format(
                  epoch,
                  i + 1,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses,
                  acc=accuracies))

    epoch_logger.log({
        'epoch': epoch,
        'loss': losses.avg,
        'acc': accuracies.avg,
        'lr': optimizer.param_groups[0]['lr']
    })

    if epoch % opt.checkpoint == 0:
        save_file_path = os.path.join(opt.result_path,
                                      'save_{}.pth'.format(epoch))
        states = {
            'epoch': epoch + 1,
            'arch': opt.arch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(states, save_file_path)

In [22]:
# import sys
# sys.path.insert(0, 'datasets/')

# from kinetics import Kinetics
# from activitynet import ActivityNet
# from ucf101 import UCF101
# from hmdb51 import HMDB51


def get_training_set(opt, spatial_transform, temporal_transform,
                     target_transform):
    assert opt.dataset in ['kinetics', 'activitynet', 'ucf101', 'hmdb51']

    if opt.dataset == 'kinetics':
        training_data = Kinetics(
            opt.video_path,
            opt.annotation_path,
            'training',
            spatial_transform=spatial_transform,
            temporal_transform=temporal_transform,
            target_transform=target_transform)
    elif opt.dataset == 'activitynet':
        training_data = ActivityNet(
            opt.video_path,
            opt.annotation_path,
            'training',
            False,
            spatial_transform=spatial_transform,
            temporal_transform=temporal_transform,
            target_transform=target_transform)
    elif opt.dataset == 'ucf101':
        training_data = UCF101(
            opt.video_path,
            opt.annotation_path,
            'training',
            spatial_transform=spatial_transform,
            temporal_transform=temporal_transform,
            target_transform=target_transform)
    elif opt.dataset == 'hmdb51':
        training_data = HMDB51(
            opt.video_path,
            opt.annotation_path,
            'training',
            spatial_transform=spatial_transform,
            temporal_transform=temporal_transform,
            target_transform=target_transform)

    return training_data


def get_validation_set(opt, spatial_transform, temporal_transform,
                       target_transform):
    assert opt.dataset in ['kinetics', 'activitynet', 'ucf101', 'hmdb51']

    if opt.dataset == 'kinetics':
        validation_data = Kinetics(
            opt.video_path,
            opt.annotation_path,
            'validation',
            opt.n_val_samples,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)
    elif opt.dataset == 'activitynet':
        validation_data = ActivityNet(
            opt.video_path,
            opt.annotation_path,
            'validation',
            False,
            opt.n_val_samples,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)
    elif opt.dataset == 'ucf101':
        validation_data = UCF101(
            opt.video_path,
            opt.annotation_path,
            'validation',
            opt.n_val_samples,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)
    elif opt.dataset == 'hmdb51':
        validation_data = HMDB51(
            opt.video_path,
            opt.annotation_path,
            'validation',
            opt.n_val_samples,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)
    return validation_data


def get_test_set(opt, spatial_transform, temporal_transform, target_transform):
    assert opt.dataset in ['kinetics', 'activitynet', 'ucf101', 'hmdb51']
    assert opt.test_subset in ['val', 'test']

    if opt.test_subset == 'val':
        subset = 'validation'
    elif opt.test_subset == 'test':
        subset = 'testing'
    if opt.dataset == 'kinetics':
        test_data = Kinetics(
            opt.video_path,
            opt.annotation_path,
            subset,
            0,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration,
            sample_stride=opt.sample_stride)
    elif opt.dataset == 'activitynet':
        test_data = ActivityNet(
            opt.video_path,
            opt.annotation_path,
            subset,
            True,
            0,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)
    elif opt.dataset == 'ucf101':
        test_data = UCF101(
            opt.video_path,
            opt.annotation_path,
            subset,
            0,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)
    elif opt.dataset == 'hmdb51':
        test_data = HMDB51(
            opt.video_path,
            opt.annotation_path,
            subset,
            0,
            spatial_transform,
            temporal_transform,
            target_transform,
            sample_duration=opt.sample_duration)

    return test_data

In [23]:
import torch
from torch.autograd import Variable
import time
import sys

from utils import AverageMeter, calculate_accuracy


def val_epoch(epoch, data_loader, model, criterion, opt, logger):
    print('validation at epoch {}'.format(epoch))

    model.eval()

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracies = AverageMeter()

    end_time = time.time()
    for i, (inputs, targets) in enumerate(data_loader):
        data_time.update(time.time() - end_time)

        if not opt.no_cuda:
            targets = targets.cuda(async=True)
        inputs = Variable(inputs, volatile=True)
        targets = Variable(targets, volatile=True)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        acc = calculate_accuracy(outputs, targets)

        losses.update(loss.data[0], inputs.size(0))
        accuracies.update(acc, inputs.size(0))

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        print('Epoch: [{0}][{1}/{2}]\t'
              'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
              'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
              'Acc {acc.val:.3f} ({acc.avg:.3f})'.format(
                  epoch,
                  i + 1,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time,
                  loss=losses,
                  acc=accuracies))

    logger.log({'epoch': epoch, 'loss': losses.avg, 'acc': accuracies.avg})

    return losses.avg

In [24]:
#m_test

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import time
import os
import sys
import json

from utils import AverageMeter


def calculate_video_results(output_buffer, video_id, test_results, class_names):
    video_outputs = torch.stack(output_buffer)
    average_scores = torch.mean(video_outputs, dim=0)
    sorted_scores, locs = torch.topk(average_scores, k=10)

    video_results = []
    for i in range(sorted_scores.size(0)):
        video_results.append({
            'label': class_names[locs[i]],
            'score': sorted_scores[i]
        })

    test_results['results'][video_id] = video_results


def test(data_loader, model, opt, class_names):
    print('test')

    model.eval()

    batch_time = AverageMeter()
    data_time = AverageMeter()

    end_time = time.time()
    output_buffer = []
    previous_video_id = ''
    test_results = {'results': {}}
    for i, (inputs, targets) in enumerate(data_loader):
        print("debug: Test:target: ",targets)
        data_time.update(time.time() - end_time)

        inputs = Variable(inputs, volatile=True)
        print("debug: Test:inputTensor.size : ",inputs.size())


        outputs = model(inputs)
        print("debug: Test: outputTensor.size : ",outputs.size())
        

        if not opt.no_softmax_in_test:
            outputs = F.softmax(outputs)
        
        for j in range(outputs.size(0)):
            if not (i == 0 and j == 0) and targets[j] != previous_video_id:
                calculate_video_results(output_buffer, previous_video_id,
                                        test_results, class_names)
                output_buffer = []
            output_buffer.append(outputs[j].data.cpu())
            previous_video_id = targets[j]

        if (i % 100) == 0:
            with open(
                    os.path.join(opt.result_path, '{}.json'.format(
                        opt.test_subset)), 'w') as f:
                json.dump(test_results, f)

        batch_time.update(time.time() - end_time)
        end_time = time.time()

        print('[{}/{}]\t'
              'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
              'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'.format(
                  i + 1,
                  len(data_loader),
                  batch_time=batch_time,
                  data_time=data_time))
    with open(
            os.path.join(opt.result_path, '{}.json'.format(opt.test_subset)),
            'w') as f:
        json.dump(test_results, f)

In [27]:
#m_main

import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

# #from opts import parse_opts
# #from model import generate_model
# #from mean import get_mean, get_std
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import LoopPadding, TemporalRandomCrop
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
# # from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger
# # from train import train_epoch
# # from validation import val_epoch
# #import test
print(__name__)
if __name__ == '__main__':
    opt = parse_opts()
    #print(opt)
    if opt.root_path != '':
        opt.video_path = os.path.join(opt.root_path, opt.video_path)
        opt.annotation_path = os.path.join(opt.root_path, opt.annotation_path)
        opt.result_path = os.path.join(opt.root_path, opt.result_path)
        if opt.resume_path:
            opt.resume_path = os.path.join(opt.root_path, opt.resume_path)
        if opt.pretrain_path:
            opt.pretrain_path = os.path.join(opt.root_path, opt.pretrain_path)
    opt.scales = [opt.initial_scale]
    for i in range(1, opt.n_scales):
        opt.scales.append(opt.scales[-1] * opt.scale_step)
    opt.arch = '{}-{}'.format(opt.model, opt.model_depth)
    opt.mean = get_mean(opt.norm_value, dataset=opt.mean_dataset)
    opt.std = get_std(opt.norm_value)
    print(opt)
    with open(os.path.join(opt.result_path, 'opts.json'), 'w') as opt_file:
        json.dump(vars(opt), opt_file)

    torch.manual_seed(opt.manual_seed)

    model, parameters = generate_model(opt)
    print(model)
    criterion = nn.CrossEntropyLoss()
    if not opt.no_cuda:
        criterion = criterion.cuda()

    if opt.no_mean_norm and not opt.std_norm:
        norm_method = Normalize([0, 0, 0], [1, 1, 1])
    elif not opt.std_norm:
        norm_method = Normalize(opt.mean, [1, 1, 1])
    else:
        norm_method = Normalize(opt.mean, opt.std)

    if not opt.no_train:
        assert opt.train_crop in ['random', 'corner', 'center']
        if opt.train_crop == 'random':
            crop_method = MultiScaleRandomCrop(opt.scales, opt.sample_size)
        elif opt.train_crop == 'corner':
            crop_method = MultiScaleCornerCrop(opt.scales, opt.sample_size)
        elif opt.train_crop == 'center':
            crop_method = MultiScaleCornerCrop(
                opt.scales, opt.sample_size, crop_positions=['c'])
        spatial_transform = Compose([
            crop_method,
            RandomHorizontalFlip(),
            ToTensor(opt.norm_value), norm_method
        ])
        temporal_transform = TemporalRandomCrop(opt.sample_duration)
        target_transform = ClassLabel()
        training_data = get_training_set(opt, spatial_transform,
                                         temporal_transform, target_transform)
        print("training data length: ", len(training_data))
        train_loader = torch.utils.data.DataLoader(
            training_data,
            batch_size=opt.batch_size,
            shuffle=True,
            num_workers=opt.n_threads,
            pin_memory=True)
        train_logger = Logger(
            os.path.join(opt.result_path, 'train.log'),
            ['epoch', 'loss', 'acc', 'lr'])
        train_batch_logger = Logger(
            os.path.join(opt.result_path, 'train_batch.log'),
            ['epoch', 'batch', 'iter', 'loss', 'acc', 'lr'])
        
        if opt.nesterov:
            dampening = 0
        else:
            dampening = opt.dampening
        optimizer = optim.SGD(
            parameters,
            lr=opt.learning_rate,
            momentum=opt.momentum,
            dampening=dampening,
            weight_decay=opt.weight_decay,
            nesterov=opt.nesterov)
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer, 'min', patience=opt.lr_patience)
    if not opt.no_val:
        spatial_transform = Compose([
            Scale(opt.sample_size),
            CenterCrop(opt.sample_size),
            ToTensor(opt.norm_value), norm_method
        ])
        temporal_transform = LoopPadding(opt.sample_duration)
        target_transform = ClassLabel()
        validation_data = get_validation_set(
            opt, spatial_transform, temporal_transform, target_transform)
        val_loader = torch.utils.data.DataLoader(
            validation_data,
            batch_size=opt.batch_size,
            shuffle=False,
            num_workers=opt.n_threads,
            pin_memory=True)
        val_logger = Logger(
            os.path.join(opt.result_path, 'val.log'), ['epoch', 'loss', 'acc'])

    if opt.resume_path:
        print('loading checkpoint {}'.format(opt.resume_path))
        checkpoint = torch.load(opt.resume_path)
        assert opt.arch == checkpoint['arch']

        opt.begin_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        if not opt.no_train:
            optimizer.load_state_dict(checkpoint['optimizer'])

    print('run')
    for i in range(opt.begin_epoch, opt.n_epochs + 1):
        if not opt.no_train:
            train_epoch(i, train_loader, model, criterion, optimizer, opt,
                        train_logger, train_batch_logger)
        if not opt.no_val:
            validation_loss = val_epoch(i, val_loader, model, criterion, opt,
                                        val_logger)

        if not opt.no_train and not opt.no_val:
            scheduler.step(validation_loss)

    if opt.test:
        spatial_transform = Compose([
            Scale(int(opt.sample_size / opt.scale_in_test)),
            CornerCrop(opt.sample_size, opt.crop_position_in_test),
            ToTensor(opt.norm_value), norm_method
        ])
        temporal_transform = LoopPadding(opt.sample_duration)
        target_transform = VideoID()

        test_data = get_test_set(opt, spatial_transform, temporal_transform,
                                 target_transform)
        test_loader = torch.utils.data.DataLoader(
            test_data,
            batch_size=opt.batch_size,
            shuffle=False,
            num_workers=opt.n_threads,
            pin_memory=True)
        test(test_loader, model, opt, test_data.class_names)

__main__
<class 'argparse.ArgumentParser'>
almost done
done
Namespace(annotation_path='data/activitynet.json', arch='resnet-10', batch_size=1, begin_epoch=1, checkpoint=10, crop_position_in_test='c', dampening=0.9, dataset='activitynet', ft_begin_index=0, initial_scale=1.0, learning_rate=0.1, lr_patience=10, manual_seed=1, mean=[114.7748, 107.7354, 99.475], mean_dataset='activitynet', model='resnet', model_depth=10, momentum=0.9, n_classes=200, n_epochs=3, n_finetune_classes=400, n_scales=5, n_threads=4, n_val_samples=3, nesterov=False, no_cuda=True, no_hflip=False, no_mean_norm=False, no_softmax_in_test=False, no_train=False, no_val=True, norm_value=1, optimizer='sgd', pretrain_path='', resnet_shortcut='B', resnext_cardinality=32, result_path='data/results', resume_path='', root_path='data', sample_duration=16, sample_size=112, scale_in_test=1.0, scale_step=0.84089641525, scales=[1.0, 0.84089641525, 0.7071067811803005, 0.5946035574934808, 0.4999999999911653], std=[38.7568578, 37.88248

__main__:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


ResNet(
  (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=Tru

KeyError: 'Traceback (most recent call last):\n  File "/Users/mbajaj/anaconda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop\n    samples = collate_fn([dataset[i] for i in batch_indices])\n  File "/Users/mbajaj/anaconda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>\n    samples = collate_fn([dataset[i] for i in batch_indices])\n  File "<ipython-input-26-6b10a0ce5cae>", line 380, in __getitem__\n    target = self.target_transform(target)\n  File "/Users/mbajaj/git_cv/3D-ResNets-PyTorch/target_transforms.py", line 20, in __call__\n    return target[\'label\']\nKeyError: \'label\'\n'

In [22]:
l = []
*l

SyntaxError: can't use starred expression here (<ipython-input-22-876fc7f1f69b>, line 2)

In [31]:
for i,data in enumerate(test_loader):
    print(i,data[1])
print(len(data))
print(data[0].shape)
print(data[1])

0 ('--1DO2V4K74',)
1 ('--1DO2V4K74',)
2 ('--1DO2V4K74',)
3 ('--1DO2V4K74',)
4 ('--1DO2V4K74',)
5 ('--1DO2V4K74',)
6 ('--1DO2V4K74',)
7 ('--1DO2V4K74',)
8 ('--1DO2V4K74',)
9 ('--1DO2V4K74',)
10 ('--1DO2V4K74',)
11 ('--1DO2V4K74',)
12 ('--1DO2V4K74',)
13 ('--1DO2V4K74',)
14 ('--1DO2V4K74',)
15 ('--1DO2V4K74',)
16 ('--1DO2V4K74',)
17 ('--1DO2V4K74',)
18 ('--1DO2V4K74',)
19 ('--1DO2V4K74',)
20 ('--1DO2V4K74',)
21 ('--1DO2V4K74',)
22 ('--1DO2V4K74',)
23 ('--1DO2V4K74',)
24 ('--1DO2V4K74',)
25 ('--1DO2V4K74',)
26 ('--1DO2V4K74',)
27 ('--1DO2V4K74',)
28 ('--1DO2V4K74',)
29 ('--1DO2V4K74',)
30 ('--1DO2V4K74',)
31 ('--1DO2V4K74',)
32 ('--1DO2V4K74',)
33 ('--1DO2V4K74',)
34 ('--1DO2V4K74',)
35 ('--1DO2V4K74',)
36 ('--1DO2V4K74',)
37 ('--1DO2V4K74',)
38 ('--1DO2V4K74',)
39 ('--1DO2V4K74',)
40 ('--1DO2V4K74',)
41 ('--1DO2V4K74',)
42 ('--1DO2V4K74',)
43 ('--1DO2V4K74',)
44 ('--1DO2V4K74',)
45 ('--1DO2V4K74',)
46 ('--1DO2V4K74',)
47 ('--1DO2V4K74',)
48 ('--1DO2V4K74',)
49 ('--1DO2V4K74',)
50 ('--1DO

In [51]:
video_path = test_loader.dataset.data[0]['video']
fps_file_path = os.path.join(video_path, 'fps')
fps_file_path
print(len(test_loader.dataset))
print(test_loader.dataset.data[0])
#print(data[1])

396
{'video': 'data/jpg_videos/v_--1DO2V4K74', 'segment': [1, 6337], 'fps': 29.96, 'video_id': '--1DO2V4K74', 'frame_indices': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}


In [40]:
samples = np.round(np.linspace(0, 99, 11))
#samples = np.linspace(0, 99, 11)


print(samples)

[ 0. 10. 20. 30. 40. 50. 59. 69. 79. 89. 99.]


In [ ]:
video_path = train_loader.dataset.data[0]['video']
# fps_file_path = os.path.join(video_path, 'fps')
# fps_file_path
print(len(train_loader.dataset))
print(train_loader.dataset.data[2])
#print(data[1])